# Preparation

In [1]:
!pip install pygame

     |████████████████████████████████| 21.8 MB 91.0 MB/s 


In [2]:
import os
os.environ['SDL_VIDEODRIVER']='dummy'
import pygame
pygame.display.set_mode((640,480))

pygame 2.1.2 (SDL 2.0.16, Python 3.7.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


<Surface(640x480x32 SW)>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Thesis_Simulation/Small eta/Avg alpha

Mounted at /content/gdrive
/content/gdrive/MyDrive/Thesis_Simulation/Small eta/Avg alpha


In [4]:
import math
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.insert(0,'../..')

from trafficSimulator import *
from SimulationAnalysis import utility_summary, compute_utility, update_utility_df

## Simulation setup

### Simulation params setup

In [5]:
all_routes = [[0, 3], [0, 1], [2, 3], [2, 1]]

v_max = 9
s0 = 2
T = 1
b_max = 2
a_max = 1

vehicle_specs = {'v_max': v_max, # Desired speed
                 's0': s0, # Safe bumper-to-bumper distance
                 'T': T, # Time gap
                 'b_max': b_max, # Deceleartion
                 'a_max': a_max, # Acceleration
                 'l' : 3
                }

vehicle_rate = 60
fast_track_factor, slow_track_factor = 1, 0.33
stop_distance = 25
fast_wait_time = 4
slow_wait_time = 4
vehicle_limit = 80

df = pd.DataFrame(columns=['Vehicle_label', 'Road_order', 
                           'Total_time', 'Leading_vehicles',
                           'Stopped_time', 'Stop_while_front'])

vehicle_preferences = dict(zip(range(vehicle_limit), 
                               [[0.25, 0.25, 0.25, 0.25]]*vehicle_limit))

### Topology setup

In [6]:
def parallel_line(current_coordinates, distance):
    """
    Returns the coordinate sets of lines that are parallel to the given line
    with the speicified distance
    
    Input:
        current_coordinates: two set of x and y coordinates that mark the start
                             and end of the given line
        distance: the distance between the given line and the parallel lines to
                  to be output
    """
    
    x1, y1 = current_coordinates[0]
    x2, y2 = current_coordinates[1]
    k = (y1-y2)/(x1-x2)
    b = y1 - k*x1
    
    cross_x = -b/k
    hypo = math.sqrt(b**2+cross_x**2)
    
    if k>0:
        new_coord_1 = ((x2+distance*abs(b)/hypo, y2-distance*abs(cross_x)/hypo),
                       (x1+distance*abs(b)/hypo, y1-distance*abs(cross_x)/hypo))

        new_coord_2 = ((x2-distance*abs(b)/hypo, y2+distance*abs(cross_x)/hypo),
                       (x1-distance*abs(b)/hypo, y1+distance*abs(cross_x)/hypo))
        
    else:
        new_coord_1 = ((x2+distance*abs(b)/hypo, y2+distance*abs(cross_x)/hypo),
                       (x1+distance*abs(b)/hypo, y1+distance*abs(cross_x)/hypo))
    
        new_coord_2 = ((x2-distance*abs(b)/hypo, y2-distance*abs(cross_x)/hypo),
                       (x1-distance*abs(b)/hypo, y1-distance*abs(cross_x)/hypo))
    
    return new_coord_1, new_coord_2

In [7]:
left, right = -75*math.sqrt(3), 75*math.sqrt(3)
bottom, top = -75, 75

left_bottom_outbound = ((left+2, 4), (-5, top-2),
                        slow_track_factor, stop_distance, slow_wait_time)
bottom_right_outbound = ((5, top-2), (right-2, 4),
                         fast_track_factor, stop_distance, fast_wait_time)

left_top_outbound = ((left+2, -4), (-5, bottom+2),
                     fast_track_factor, stop_distance, fast_wait_time)
top_right_outbound = ((5, bottom+2), (right-2, -4),
                      slow_track_factor, stop_distance, slow_wait_time)

bottom_left_inbound = parallel_line(current_coordinates=left_bottom_outbound, 
                                    distance=2.5)[0]
right_bottom_inbound = parallel_line(current_coordinates=bottom_right_outbound, 
                                     distance=2.5)[1]

top_left_inbound = parallel_line(current_coordinates=left_top_outbound, 
                                 distance=2.5)[0]
right_top_inbound = parallel_line(current_coordinates=top_right_outbound, 
                                  distance=2.5)[1]

## For the connections of the topology, we can either represent with concrete lines
## or just as a dot (as shown below). Either case, the vehicles will not actually go 
## pass these routes; they are more for demonstration purposes

connection_top_bottom = ((-1.25, bottom+2), (-1.25, top-2))
connection_bottom_top = ((1.25, top-2), (1.25, bottom+2))

### Utility params setup

In [8]:
# Smaller delta encourages more exploration
delta = 0.25

# Penalty proportion for delay caused
alpha = 5

# # Interpolation factor between money and time
# gamma_mean, gamma_var = 5, 0.5

# # Level of risk aversion
# eta_mean, eta_var = 0.5, 0.1

## Function needed for running simulation

In [9]:
 def run_simulation(round_number, all_routes, vehicle_limit, df, vehicle_preferences,
                    vehicle_rate, vehicle_specs):
    """
    Executes the simulation and returns certain data of the simulation.
    
    Input:
        round_number (integer): the number of the current simulation
        all_routes (list): all possible paths that a vehicle can take
        vehicle_limit (integer): the total number of vehicles in the simulation
        df (pandas DataFrame): a dataframe for storing data from the simulation
        vehicle_preferences (dict): store vehicle label as keys and their probabilities
                                    of choosing each of the possible routes as values
        vehicle_rate (integer): frequency of generating new vehicles
        vehicle_specs (dict): parameters related to the vehicles
    """
    global left_bottom_outbound
    global bottom_right_outbound
    global left_top_outbound
    global top_right_outbound
    global connection_top_bottom
    global connection_bottom_top
    global bottom_left_inbound
    global right_bottom_inbound
    global top_left_inbound
    global right_top_inbound
    
    records = df.copy()
    sim = Simulation({
        'round_number': round_number,
        'all_routes': all_routes, # All possible (and reasonable) routes
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'records': records, # Table that will capture the needed vehicle-related info
        'vehicle_preferences': vehicle_preferences
        # A dictionary that stores the vehicles' labels as keys and their list of probabilities
        # of choosing each possible route as values
        # For the initial few rounds all vehicles choose all routes with equal weights.
        # For more complicated topologies some other algorithms will be needed as all possible
        # routes are not as explicit as in here.
        })

    sim.create_roads([
        ## Key routes
        left_bottom_outbound, # Road #0
        bottom_right_outbound, # Road #1

        left_top_outbound, # Road #2
        top_right_outbound, # Road #3

        connection_top_bottom, # Road #4
        connection_bottom_top, # Road #5

        ## Curved corners
        # Note: in the simulation, the vehicles will not actually go pass these
        # routes; they are more for aesthetic purposes

        # Outbound corners
        *curve_road(left_bottom_outbound[1], 
                    (bottom_right_outbound[0][0], bottom_right_outbound[0][1]+0.01), 
                    (0, top), 16), # Outbound bottom corner

        *curve_road(left_top_outbound[1], 
                    (top_right_outbound[0][0], top_right_outbound[0][1]+0.01), 
                    (0, bottom), 16), # Outbound top corner

        *curve_road(left_bottom_outbound[0], 
                    (left_top_outbound[0][0]+0.01, left_top_outbound[0][1]), 
                    (left, 0), 16), # Outbound left corner

        *curve_road(bottom_right_outbound[1], 
                    (top_right_outbound[1][0]+0.01, top_right_outbound[1][1]), 
                    (right, 0), 16), # Outbound right corner
    ])


    sim.create_gen({
        'vehicle_rate': vehicle_rate, # Rate of generating new vehicles
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'vehicles': vehicle_specs
        })

    # Start simulation
    win = Window(sim)
    win.zoom = 4
    new_records = win.run(steps_per_update=5)
    
    return new_records

# Official start: 1st round

In [10]:
%%time
# Run the 1st round
record_df1 = run_simulation(1, all_routes, vehicle_limit, df, vehicle_preferences,
                             vehicle_rate, vehicle_specs)

CPU times: user 3min 31s, sys: 5.87 s, total: 3min 37s
Wall time: 3min 35s


In [11]:
record_df1

,Vehicle_label,Road_order,Total_time,Leading_vehicles,Stopped_time,Stop_while_front
0,0,"[0, 3]",97.9,"[-999, 10]","[0, 0]","[0, 0]"
1,1,"[0, 3]",109.9,"[0, 14]","[3.049999999999997, 0]","[3.049999999999997, 0]"
2,2,"[0, 1]",78.6,"[1, -999]","[0, 0]","[0, 0]"
3,3,"[2, 1]",36.4,"[-999, -999]","[0, 0]","[0, 0]"
4,4,"[2, 1]",38.1,"[3, 3]","[0, 0]","[0, 0]"
...,...,...,...,...,...,...
75,75,"[2, 1]",160.4,"[72, 62]","[17.600000000000115, 0]","[0, 0]"
76,76,"[0, 3]",180.5,"[74, 73]","[7.549999999999981, 0]","[0, 0]"
77,77,"[2, 3]",194.65,"[75, 63]","[21.050000000000164, 0]","[0, 0]"
78,78,"[0, 3]",176.6,"[76, 76]","[5.799999999999987, 0]","[0, 0]"


## Simulation Analysis of the 1st round

In [12]:
record_df1['Round_number'] = 1

record_df = record_df1.copy()
record_df.head(10)

,Vehicle_label,Road_order,Total_time,Leading_vehicles,Stopped_time,Stop_while_front,Round_number
0,0,"[0, 3]",97.9,"[-999, 10]","[0, 0]","[0, 0]",1
1,1,"[0, 3]",109.9,"[0, 14]","[3.049999999999997, 0]","[3.049999999999997, 0]",1
2,2,"[0, 1]",78.6,"[1, -999]","[0, 0]","[0, 0]",1
3,3,"[2, 1]",36.4,"[-999, -999]","[0, 0]","[0, 0]",1
4,4,"[2, 1]",38.1,"[3, 3]","[0, 0]","[0, 0]",1
5,5,"[2, 1]",39.9,"[4, 4]","[0, 0]","[0, 0]",1
6,6,"[2, 1]",41.8,"[5, 5]","[0, 0]","[0, 0]",1
7,7,"[0, 3]",111.7,"[2, 15]","[0, 0]","[0, 0]",1
8,8,"[2, 1]",43.55,"[6, 6]","[0, 0]","[0, 0]",1
9,9,"[0, 1]",81.6,"[7, 20]","[0, 0]","[0, 0]",1


### Set up dataframe

In [13]:
utility_df = pd.read_csv("../Zero alpha/Saved_data/Utility_data/Round_1.csv")

utility_df['Vehicle_label'] = list(range(len(record_df.Vehicle_label.unique())))
utility_df['Routes_taken'] = [[]] * len(utility_df)
utility_df['Utilities'] = [[0] * len(all_routes)] * len(utility_df)
utility_df['Probabilities'] = [[0] * len(all_routes)] * len(utility_df)

utility_df = utility_df.sort_values('Vehicle_label', ascending=True).reset_index(drop=True)            
utility_df.head(5)

,Vehicle_label,Utilities,Probabilities,Routes_taken,Gamma,Eta
0,0,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],5.661216,0.489242
1,1,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],4.433706,0.522380
2,2,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],5.244867,0.439327
3,3,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],4.627887,0.443460
4,4,"[0, 0, 0, 0]","[0, 0, 0, 0]",[],4.259416,0.435075


## Summary of the 1st round

In [14]:
utility_tmp = utility_summary(all_routes, record_df)
utility_df = update_utility_df(all_routes,
                               delta, 
                               alpha, 
                               record_df=record_df,
                               df=utility_df, 
                               utility_tmp=utility_tmp)

display(utility_tmp.head(10))
display(utility_df.head(10))

,Vehicle_label,Road_order,Caused_delay,Count,Total_time
0,0,0,3.05,1,97.90
1,1,0,0.00,1,109.90
2,2,1,0.00,1,78.60
3,3,3,0.00,1,36.40
4,4,3,0.00,1,38.10
5,5,3,0.00,1,39.90
6,6,3,0.00,1,41.80
7,7,0,0.00,1,111.70
8,8,3,0.00,1,43.55
9,9,1,0.00,1,81.60


,Vehicle_label,Utilities,Probabilities,Routes_taken,Gamma,Eta
0,0,"[-1.6860590298803049, 16.651092223153956, 16.6...","[3.6237242365575082e-09, 0.3333333321254252, 0...",[0],5.661216,0.489242
1,1,"[-0.31966600267137046, 16.651092223153956, 16....","[1.4209280510989285e-08, 0.3333333285969065, 0...",[0],4.433706,0.522380
2,2,"[16.651092223153956, -0.05750199728632599, 16....","[0.3333333271772114, 1.846836575728151e-08, 0....",[1],5.244867,0.439327
3,3,"[16.651092223153956, 16.651092223153956, 16.65...","[0.33333332699354656, 0.33333332699354656, 0.3...",[3],4.627887,0.443460
4,4,"[16.651092223153956, 16.651092223153956, 16.65...","[0.33333332684655415, 0.33333332684655415, 0.3...",[3],4.259416,0.435075
5,5,"[16.651092223153956, 16.651092223153956, 16.65...","[0.33333332780881414, 0.33333332780881414, 0.3...",[3],5.069627,0.490281
6,6,"[16.651092223153956, 16.651092223153956, 16.65...","[0.33333332730311016, 0.33333332730311016, 0.3...",[3],5.343152,0.456743
7,7,"[-0.19602490365599734, 16.651092223153956, 16....","[1.6079359090314643e-08, 0.3333333279735469, 0...",[0],4.797432,0.483636
8,8,"[16.651092223153956, 16.651092223153956, 16.65...","[0.3333333299364597, 0.3333333299364597, 0.333...",[3],5.318841,0.624211
9,9,"[16.651092223153956, 0.11343737638874707, 16.6...","[0.3333333260295944, 2.1911216608315888e-08, 0...",[1],5.279262,0.369614


In [15]:
# Export
record_df.to_csv("Saved_data/Simulation_records/Round_1.csv", index=False)
utility_df.to_csv("Saved_data/Utility_data/Round_1.csv", index=False)

# Run the remaining rounds
Utilities, probabilities are updated after each round and fed into the next round

In [16]:
%%time

import time
time_l = []
begin = time.time()

# Updated probabilities for the 2nd round:
vehicle_preferences = dict(zip(range(vehicle_limit), utility_df.Probabilities.to_list()))

rounds = 100

for s in range(2, rounds+1):
    record_tmp = run_simulation(s, all_routes, 
                                vehicle_limit, 
                                df, vehicle_preferences,
                                vehicle_rate, vehicle_specs)
    record_tmp['Round_number'] = s
    
    record_df = pd.concat([record_df, record_tmp], ignore_index=True)
    
    utility_tmp = utility_summary(all_routes, record_df)
    utility_df = update_utility_df(all_routes,
                                   delta, 
                                   alpha, 
                                   record_df=record_df,
                                   df=utility_df, 
                                   utility_tmp=utility_tmp,
                                   round_number=s)
    
    vehicle_preferences = dict(zip(range(vehicle_limit), 
                                   utility_df.Probabilities.to_list()))
    
    # Export
    record_df.to_csv(f"Saved_data/Simulation_records/Round_{s}.csv", index=False)
    utility_df.to_csv(f"Saved_data/Utility_data/Round_{s}.csv", index=False)
    
    time_l.append(round(time.time()-begin-sum(time_l), 2))

CPU times: user 5h 53min 11s, sys: 9min 45s, total: 6h 2min 56s
Wall time: 5h 49min 37s


In [17]:
utility_tmp.to_csv(f"Saved_data/Simulation_records/Summary.csv", index=False)